<a href="https://colab.research.google.com/github/Tannishtharoyc/attendence_automation-pynb.ipynb/blob/main/ATTENDENCE_AUTOMATION_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 983.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=8f319f01dbe882c0b068ff2973ae4941413efd2fd985fb26714e8759dd535407
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from deepface import DeepFace

24-06-21 14:38:04 - Directory /root/.deepface created
24-06-21 14:38:04 - Directory /root/.deepface/weights created


In [ ]:
import zipfile
import os

# Path to the uploaded zip file
zip_file_path = '/content/Training Images-20240623T213230Z-001.zip'
#extract_path = 'path/to/extracted/dataset/'
# Unzip the file
with zipfile.ZipFile(zip_file_path,'r') as zip:
  zip.extractall()
  print('Done')
#with zipfile(zip_file_path, 'r') as zip_ref:
    #zip_ref.extractall()

# List the directories to check the structure
extracted_dirs = os.listdir()
print(extracted_dirs)

Done
['.config', 'Training Images-20240623T213230Z-001.zip', 'drive', 'Training Images', 'sample_data']


In [ ]:
import cv2
import numpy as np

def preprocess_image_for_facenet(image):
    """
    Preprocess the given image for the FaceNet model.

    Parameters:
    image (numpy.ndarray): Input image to be preprocessed.

    Returns:
    numpy.ndarray: Preprocessed image or None if the input is invalid.
    """
    if image is None or image.size == 0:
        return None

    # Resize image to 160x160 as expected by the FaceNet model
    resized_image = cv2.resize(image, (160, 160))

    # Normalize pixel values to the range [-1, 1]
    normalized_image = (resized_image - 127.5) / 127.5

    return normalized_image


In [ ]:
import os
import cv2
import numpy as np

def load_images_from_directory(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            try:
                with Image.open(img_path) as img:
                    img = img.resize((128, 128))  # Resize the image to a fixed size (optional)
                    img_array = np.array(img)
                    images.append(img_array)
                    # Assuming labels are part of the filename, e.g., 'cat.1.jpg'
                    label = filename.split('.')[0]
                    labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)




In [ ]:
def get_face_embedding(image):
    if image is not None:
        temp_image_path = 'temp_image.jpg'
        cv2.imwrite(temp_image_path, (image * 127.5 + 127.5).astype(np.uint8))  # Convert back to original pixel range
        embedding = DeepFace.represent(img_path=temp_image_path, model_name="Facenet", enforce_detection=False)[0]["embedding"]
        os.remove(temp_image_path)
        return embedding
    else:
        return None

def recognize_face(test_embedding, known_face_encodings, known_face_names):
    distances = [np.linalg.norm(test_embedding - known_face_encoding) for known_face_encoding in known_face_encodings]
    best_match_index = np.argmin(distances)
    if distances[best_match_index] < 0.6:
        return known_face_names[best_match_index]
    else:
        return "Unknown"

def monofacetest(test_directory, known_face_encodings, known_face_names):
    for filename in os.listdir(test_directory):
        test_image_path = os.path.join(test_directory, filename)
        test_image = cv2.imread(test_image_path)
        test_image_preprocessed = preprocess_image_for_facenet(test_image)
        if test_image_preprocessed is not None:
            test_embedding = get_face_embedding(test_image_preprocessed)
            if test_embedding is not None:
                name = recognize_face(test_embedding, known_face_encodings, known_face_names)
                plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
                plt.title(f"Person name: {name}")
                plt.axis('off')
                plt.show()


In [ ]:
def draw_faces_on_image(image, faces, known_face_encodings, known_face_names):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax = plt.gca()

    for face in faces:
        facial_area = face['facial_area']
        x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
        face_image = image[y:y+h, x:x+w]
        face_preprocessed = preprocess_image_for_facenet(face_image)

        if face_preprocessed is not None:
            face_embedding = get_face_embedding(face_preprocessed)
            if face_embedding is not None:
                name = recognize_face(face_embedding, known_face_encodings, known_face_names)
                ax.add_patch(plt.Rectangle((x, y), w, h, edgecolor='red', facecolor='none', linewidth=2))
                plt.text(x, y - 10, name, color='red', fontsize=12, weight='bold')

    plt.axis('off')
    plt.show()

def multifacetest(test_directory, known_face_encodings, known_face_names):
    for filename in os.listdir(test_directory):
        test_image_path = os.path.join(test_directory, filename)
        test_image = cv2.imread(test_image_path)

        faces = DeepFace.extract_faces(img_path=test_image_path, detector_backend='opencv', enforce_detection=False)
        draw_faces_on_image(test_image, faces, known_face_encodings, known_face_names)


In [3]:
# Load training images
training_dir = r'/content/drive/MyDrive/Directory /Training Images'
train_images, train_labels = load_images_from_directory(training_dir)

NameError: name 'load_images_from_directory' is not defined

In [ ]:
!pip install face_recognition opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=0765b0f6a53fae06c3c624cfce67fdeb42a2d349ed154a81089e6ecc0478d67e
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
!pip uninstall -y dlib face_recognition

Found existing installation: dlib 19.22.1
Uninstalling dlib-19.22.1:
  Successfully uninstalled dlib-19.22.1
Found existing installation: face-recognition 1.3.0
Uninstalling face-recognition-1.3.0:
  Successfully uninstalled face-recognition-1.3.0


In [ ]:
!pip install dlib==19.22.1
!pip install face_recognition
!pip install opencv-python

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw

def load_image(file_path):
    return cv2.imread(file_path)

def encode_faces(image, face_locations):
    encodings = []
    for (top, right, bottom, left) in face_locations:
        face = image[top:bottom, left:right]
        face = cv2.resize(face, (128, 128))  # Resize to 128x128 for simplicity
        encoding = face.flatten()  # Flatten the image array to create a simple encoding
        encodings.append(encoding)
    return encodings

def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return [(y, x+w, y+h, x) for (x, y, w, h) in faces]

In [4]:
def monofacetest(directory, known_face_encodings, known_face_names):
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            image = load_image(img_path)
            face_locations = detect_faces(image)
            face_encodings = encode_faces(image, face_locations)

            pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(pil_image)

            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                matches = [np.linalg.norm(known_face_encoding - face_encoding) < 10000 for known_face_encoding in known_face_encodings]  # Simplified matching
                name = "Unknown"

                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]

                draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))
                draw.text((left + 6, bottom - 6), name, fill=(255, 255, 255, 255))

            del draw

            pil_image.show()

# Example usage
monoface_detection = r'/content/drive/MyDrive/Directory /Testing Images'

# Load known face images and encode them
try:
    known_image_1 = load_image("/path/to/known_person_1.jpg")
    known_face_locations_1 = detect_faces(known_image_1)
    known_face_encodings_1 = encode_faces(known_image_1, known_face_locations_1)[0]

    known_image_2 = load_image("/path/to/known_person_2.jpg")
    known_face_locations_2 = detect_faces(known_image_2)
    known_face_encodings_2 = encode_faces(known_image_2, known_face_locations_2)[0]

    known_face_encodings = [
        known_face_encodings_1,
        known_face_encodings_2
    ]

    known_face_names = [
        "Person 1",
        "Person 2"
        "Person 3"
        "Person 4"
        "Person 5"
    ]

    monofacetest(monoface_detection, known_face_encodings, known_face_names)

except FileNotFoundError as e:
    print(f"Error loading image: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: name 'load_image' is not defined


In [7]:
pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 9.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=2a36bcb4d760f654985edb7058f235707fcd7e3d722b59de423d5f7789dc1b8a
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [1]:
import os
import cv2
from deepface import DeepFace

# Load known faces and their names
known_faces = {
"Tannishtha Roy Chowdhury": "/content/drive/MyDrive/Directory /Training Images/Tannishtha Roy Chowdhury /IMG_6218.JPG",
    "Shreya Singha Roy": "/content/drive/MyDrive/Directory /Training Images/Shreya Singha Roy /27124ff3-912a-44b5-b5cd-68a6a6a0f426.jpg",
    "Debarati Sarkar": "/content/drive/MyDrive/Directory /Training Images/Debarati Sarkar/20626469-8538-4e6e-93d0-48e48764ab71.jpg",
                "Aishi Sarkar": "/content/drive/MyDrive/Directory /Training Images/Aishi Sarkar/IMG-20240604-WA0046.jpg",
                "Om Sarkar": "/content/drive/MyDrive/Directory /Training Images/Om Sarkar/Img-20240604-WA0035.jpg",
                }

def multifacetest(image_directory, known_faces):
    for image_name in os.listdir(image_directory):
        image_path = os.path.join(image_directory, image_name)

        if not os.path.isfile(image_path):
            print(f"Error loading {image_name}: File does not exist")
            continue

        try:
            # Detect and analyze faces for emotions
            results = DeepFace.analyze(img_path=image_path, actions=['emotion'], enforce_detection=False)

            if not results:
                print(f"No faces found in {image_name}")
                continue

            # Extract faces from results
            face_results = results if isinstance(results, list) else [results]

            for face_result in face_results:
                found_match = False
                dominant_emotion = face_result.get('dominant_emotion', 'Unknown')

                for name, known_image_path in known_faces.items():
                    if not os.path.isfile(known_image_path):
                        print(f"Known image path for {name} does not exist: {known_image_path}")
                        continue

                    # Verify the detected face against the known face
                    result = DeepFace.verify(img1_path=image_path, img2_path=known_image_path, enforce_detection=False)
                    if result["verified"]:
                        print(f"Detected {name} in {image_name} with emotion: {dominant_emotion}")
                        found_match = True
                        break

                if not found_match:
                    print(f"No matches found for a face in {image_name}, but detected emotion: {dominant_emotion}")

        except Exception as e:
            print(f"Error processing {image_name}: {str(e)}")


In [2]:
# Directory containing the test images
monoface_detection = r'/content/drive/MyDrive/Directory /Testing Images'
multifacetest(monoface_detection, known_faces)

Detected Debarati Sarkar in 0e82c9f5-be0d-440a-a004-79680eaa8dac.jpg with emotion: happy
Detected Debarati Sarkar in 0e82c9f5-be0d-440a-a004-79680eaa8dac.jpg with emotion: happy
Detected Debarati Sarkar in b22eb268-5485-4e3c-92f5-6f3a12d7a0fe.jpg with emotion: happy
Detected Debarati Sarkar in b22eb268-5485-4e3c-92f5-6f3a12d7a0fe.jpg with emotion: neutral
Detected Debarati Sarkar in b22eb268-5485-4e3c-92f5-6f3a12d7a0fe.jpg with emotion: surprise
Detected Debarati Sarkar in 9c36b6c5-8fa5-4b87-a517-244e9dc344ef.jpg with emotion: happy
Detected Debarati Sarkar in 9c36b6c5-8fa5-4b87-a517-244e9dc344ef.jpg with emotion: happy
Detected Debarati Sarkar in 4611d3ef-16cf-40f8-93c6-cc78dd70720e.jpg with emotion: happy
Detected Debarati Sarkar in 4611d3ef-16cf-40f8-93c6-cc78dd70720e.jpg with emotion: happy
Detected Debarati Sarkar in 4611d3ef-16cf-40f8-93c6-cc78dd70720e.jpg with emotion: fear
Detected Tannishtha Roy Chowdhury in 1000221691.webp with emotion: happy
Detected Tannishtha Roy Chowdhury

In [3]:
import os
import cv2
from deepface import DeepFace

# Load known faces and their names
known_faces = {
    "Tannishtha Roy Chowdhury": "/content/drive/MyDrive/Directory /Training Images/Tannishtha Roy Chowdhury /IMG_6218.JPG",
    "Shreya Singha Roy": "/content/drive/MyDrive/Directory /Training Images/Shreya Singha Roy /27124ff3-912a-44b5-b5cd-68a6a6a0f426.jpg",
    "Debarati Sarkar": "/content/drive/MyDrive/Directory /Training Images/Debarati Sarkar/20626469-8538-4e6e-93d0-48e48764ab71.jpg",
                "Aishi Sarkar": "/content/drive/MyDrive/Directory /Training Images/Aishi Sarkar/IMG-20240604-WA0046.jpg",
                "Om Sarkar": "/content/drive/MyDrive/Directory /Training Images/Om Sarkar/Img-20240604-WA0035.jpg",
                }

def multifacetest(image_directory, known_faces):
    for image_name in os.listdir(image_directory):
        image_path = os.path.join(image_directory, image_name)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading {image_name}: Invalid image file")
            continue

        try:
            # Detect and analyze faces for emotions
            results = DeepFace.analyze(img_path=image_path, actions=['emotion'], enforce_detection=False)

            if not results:
                print(f"No faces found in {image_name}")
                continue

            # Extract faces from results
            face_results = results if isinstance(results, list) else [results]

            for face_result in face_results:
                found_match = False
                dominant_emotion = face_result['dominant_emotion']

                for name, known_image_path in known_faces.items():
                    if not os.path.isfile(known_image_path):
                        print(f"Known image path for {name} does not exist: {known_image_path}")
                        continue

                    # Verify the detected face against the known face
                    result = DeepFace.verify(img1_path=image_path, img2_path=known_image_path, enforce_detection=False)
                    if result["verified"]:
                        print(f"Detected {name} in {image_name} with emotion: {dominant_emotion}")
                        found_match = True
                        break

                if not found_match:
                    print(f"No matches found for a face in {image_name}, but detected emotion: {dominant_emotion}")

        except Exception as e:
            print(f"Error processing {image_name}: {str(e)}")



In [4]:
# Directory containing the test images
multiface_detection = r'/content/drive/MyDrive/Directory /Testing Images'
multifacetest(multiface_detection, known_faces)

Detected Debarati Sarkar in 0e82c9f5-be0d-440a-a004-79680eaa8dac.jpg with emotion: happy
Detected Debarati Sarkar in 0e82c9f5-be0d-440a-a004-79680eaa8dac.jpg with emotion: happy
Detected Debarati Sarkar in b22eb268-5485-4e3c-92f5-6f3a12d7a0fe.jpg with emotion: happy
Detected Debarati Sarkar in b22eb268-5485-4e3c-92f5-6f3a12d7a0fe.jpg with emotion: neutral
Detected Debarati Sarkar in b22eb268-5485-4e3c-92f5-6f3a12d7a0fe.jpg with emotion: surprise
Detected Debarati Sarkar in 9c36b6c5-8fa5-4b87-a517-244e9dc344ef.jpg with emotion: happy
Detected Debarati Sarkar in 9c36b6c5-8fa5-4b87-a517-244e9dc344ef.jpg with emotion: happy
Detected Debarati Sarkar in 4611d3ef-16cf-40f8-93c6-cc78dd70720e.jpg with emotion: happy
Detected Debarati Sarkar in 4611d3ef-16cf-40f8-93c6-cc78dd70720e.jpg with emotion: happy
Detected Debarati Sarkar in 4611d3ef-16cf-40f8-93c6-cc78dd70720e.jpg with emotion: fear
Detected Tannishtha Roy Chowdhury in 1000221691.webp with emotion: happy
Detected Tannishtha Roy Chowdhury